# 情感分析：使用卷积神经网络


在 :numref:`chap_cnn`中，我们探讨了使用二维卷积神经网络处理二维图像数据的机制，并将其应用于局部特征，如相邻像素。虽然卷积神经网络最初是为计算机视觉设计的，但它也被广泛用于自然语言处理。简单地说，只要将任何文本序列想象成一维图像即可。通过这种方式，一维卷积神经网络可以处理文本中的局部特征，例如$n$元语法。

本节将使用*textCNN*模型来演示如何设计一个表示单个文本 :cite:`Kim.2014`的卷积神经网络架构。与 :numref:`fig_nlp-map-sa-rnn`中使用带有GloVe预训练的循环神经网络架构进行情感分析相比， :numref:`fig_nlp-map-sa-cnn`中唯一的区别在于架构的选择。

![将GloVe放入卷积神经网络架构进行情感分析](../img/nlp-map-sa-cnn.svg)
:label:`fig_nlp-map-sa-cnn`


In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 64
train_iter, test_iter, vocab = d2l.load_data_imdb(batch_size)

## 一维卷积

在介绍该模型之前，让我们先看看一维卷积是如何工作的。请记住，这只是基于互相关运算的二维卷积的特例。

![一维互相关运算。阴影部分是第一个输出元素以及用于输出计算的输入和核张量元素：$0\times1+1\times2=2$](../img/conv1d.svg)
:label:`fig_conv1d`

如 :numref:`fig_conv1d`中所示，在一维情况下，卷积窗口在输入张量上从左向右滑动。在滑动期间，卷积窗口中某个位置包含的输入子张量（例如， :numref:`fig_conv1d`中的$0$和$1$）和核张量（例如， :numref:`fig_conv1d`中的$1$和$2$）按元素相乘。这些乘法的总和在输出张量的相应位置给出单个标量值（例如， :numref:`fig_conv1d`中的$0\times1+1\times2=2$）。

我们在下面的`corr1d`函数中实现了一维互相关。给定输入张量`X`和核张量`K`，它返回输出张量`Y`。


In [ ]:
def corr1d(X, K):
    steps = X.shape[0] - K.shape[0] + 1
    # 输出Y的长度为X的长度减去K的长度加1
    Y = torch.zeros((steps))
    for i in range(Y.shape[0]):
        # 计算X的第i个元素到第i+K.shape[0]个元素与K的卷积
        Y[i] = (X[i: i + K.shape[0]] * K).sum()
    return Y

我们可以从 :numref:`fig_conv1d`构造输入张量`X`和核张量`K`来验证上述一维互相关实现的输出。


In [ ]:
X, K = torch.tensor([0, 1, 2, 3, 4, 5, 6]), torch.tensor([1, 2])
corr1d(X, K)

对于任何具有多个通道的一维输入，卷积核需要具有相同数量的输入通道。然后，对于每个通道，对输入的一维张量和卷积核的一维张量执行互相关运算，将所有通道上的结果相加以产生一维输出张量。 :numref:`fig_conv1d_channel`演示了具有3个输入通道的一维互相关操作。

![具有3个输入通道的一维互相关运算。阴影部分是第一个输出元素以及用于输出计算的输入和核张量元素：$2\times(-1)+3\times(-3)+1\times3+2\times4+0\times1+1\times2=2$](../img/conv1d-channel.svg)
:label:`fig_conv1d_channel`

我们可以实现多个输入通道的一维互相关运算，并在 :numref:`fig_conv1d_channel`中验证结果。


In [ ]:
def corr1d_multi_in(X, K):
    outs = []
    for x, k in zip(X, K):
        outs.append(corr1d(x, k))
    return outs
    
    


In [ ]:
X = torch.tensor([[2, 3, 4, 5, 6, 7, 8],
                  [1, 2, 3, 4, 5, 6, 7],
                  [0, 1, 2, 3, 4, 5, 6]])
              
K = torch.tensor([[-1, -3],
                  [3, 4], 
                  [1, 2]])
                  
print(corr1d_multi_in(X, K))
print(sum(corr1d_multi_in(X, K)))

注意，多输入通道的一维互相关等同于单输入通道的二维互相关。举例说明， :numref:`fig_conv1d_channel`中的多输入通道一维互相关的等价形式是 :numref:`fig_conv1d_2d`中的单输入通道二维互相关，其中卷积核的高度必须与输入张量的高度相同。

![具有单个输入通道的二维互相关操作。阴影部分是第一个输出元素以及用于输出计算的输入和内核张量元素： $2\times(-1)+3\times(-3)+1\times3+2\times4+0\times1+1\times2=2$](../img/conv1d-2d.svg)
:label:`fig_conv1d_2d`

 :numref:`fig_conv1d`和 :numref:`fig_conv1d_channel`中的输出都只有一个通道。与 :numref:`subsec_multi-output-channels`中描述的具有多个输出通道的二维卷积相同，我们也可以为一维卷积指定多个输出通道。


In [ ]:
def corr1d_multi_in_2(X, K):
    steps = X.shape[1] - K.shape[1] + 1

    Y = torch.zeros((steps))
    for i in range(steps):
        Y[i] = (X[:, i:i + K.shape[1]] * K).sum()
    return Y
    

In [ ]:

X = torch.tensor([[2, 3, 4, 5, 6, 7, 8],
                  [1, 2, 3, 4, 5, 6, 7],
                  [0, 1, 2, 3, 4, 5, 6]])
              
K = torch.tensor([[-1, -3],
                  [3, 4], 
                  [1, 2]])

corr1d_multi_in_2(X, K)


## 最大时间汇聚层

类似地，我们可以使用汇聚层从序列表示中提取最大值，作为跨时间步的最重要特征。textCNN中使用的*最大时间汇聚层*的工作原理类似于一维全局汇聚 :cite:`Collobert.Weston.Bottou.ea.2011`。对于每个通道在不同时间步存储值的多通道输入，每个通道的输出是该通道的最大值。请注意，最大时间汇聚允许在不同通道上使用不同数量的时间步。

## textCNN模型

使用一维卷积和最大时间汇聚，textCNN模型将单个预训练的词元表示作为输入，然后获得并转换用于下游应用的序列表示。

(B,Q,E)->(Q,B,E)=(n,1,d)

对于具有由$d$维向量表示的$n$个词元的单个文本序列，输入张量的宽度、高度和通道数分别为$n$、$1$和$d$。textCNN模型将输入转换为输出，如下所示：

1. 定义多个一维卷积核，并分别对输入执行卷积运算。具有不同宽度的卷积核可以捕获不同数目的相邻词元之间的局部特征。
1. 在所有输出通道上执行最大时间汇聚层，然后将所有标量汇聚输出连结为向量。
1. 使用全连接层将连结后的向量转换为输出类别。Dropout可以用来减少过拟合。

![textCNN的模型架构](../img/textcnn.svg)
:label:`fig_conv1d_textcnn`

 :numref:`fig_conv1d_textcnn`通过一个具体的例子说明了textCNN的模型架构。输入是具有11个词元的句子，其中每个词元由6维向量表示。因此，我们有一个宽度为11的6通道输入。定义两个宽度为2和4的一维卷积核，分别具有4个和5个输出通道。它们产生4个宽度为$11-2+1=10$的输出通道和5个宽度为$11-4+1=8$的输出通道。尽管这9个通道的宽度不同，但最大时间汇聚层给出了一个连结的9维向量，该向量最终被转换为用于二元情感预测的2维输出向量。

【注】虽然文本通常不说“通道”，但在实现时（如 PyTorch），词嵌入维度会被视为“通道维”，以便使用标准 CNN 操作。

【注】输出通道数 = 卷积核的数量（filter 数量）

 尽管所有卷积核都作用于相同的输入张量，每个卷积核通过学习能够捕捉到输入数据中不同的局部特征。例如，在图像处理领域，一个卷积核可能学习到识别边缘，另一个可能专注于纹理，而第三个可能擅长捕捉特定形状等。

经过最大汇聚之后， 这个 9 维向量的意义是什么？
* 它是模型从不同 n-gram 尺度（bi-gram、4-gram 等）中提取出的最强局部特征响应。
* 每一维代表“某种语义模式是否在句子中强烈出现”。
* 后续通常接一个 全连接层（FC）+ softmax，用于分类（如正面/负面情感）。

### 定义模型

我们在下面的类中实现textCNN模型。与 :numref:`sec_sentiment_rnn`的双向循环神经网络模型相比，除了用卷积层代替循环神经网络层外，我们还使用了两个嵌入层：一个是可训练权重，另一个是固定权重。


In [ ]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, pretrained_embeddings, embed_size, kernel_sizes, num_channels, **kwargs):
        super(TextCNN, self).__init__(**kwargs)

        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 这个嵌入层不需要训练
        # 通常用于加载预训练词向量（如 GloVe、Word2Vec）并冻结其参数（即不更新）
        # 这种双嵌入设计是 Kim (2014) 提出的经典 TextCNN 的一种变体：一个可微调，一个固定，增强泛化能力。
        self.constant_embedding = nn.Embedding(vocab_size, embed_size)

        # 如果提供了预训练词向量，加载并冻结
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings) # 用于训练
            self.constant_embedding.weight.data.copy_(pretrained_embeddings) # 用于固定
            self.constant_embedding.weight.requires_grad = False
        else:
            # 如果没有预训练向量，则也设为不可训练（与原代码一致）
            self.constant_embedding.weight.requires_grad = False

        # 创建多个一维卷积层
        self.convs = nn.ModuleList()
        # 100,3x3
        # 100,4x4
        # 100,5x5
        for c, k in zip(num_channels, kernel_sizes):
            # 输入通道数是 2*E concatenate 了两个嵌入层的输出）
            # 输出通道数是 c（即每个卷积核的数量）
            # 卷积核大小是 k（即卷积窗口的宽度），一维
            self.convs.append(nn.Conv1d(2 * embed_size, c, k))
        

        
        # 最大时间汇聚层没有参数，因此可以共享此实例
        self.pool = nn.AdaptiveMaxPool1d(1)

        # 激活函数
        self.relu = nn.ReLU()


        # 在全连接层前加 dropout（防止过拟合），丢弃率 50%
        self.dropout = nn.Dropout(0.5)
        # 分类器
        # 最终的线性分类器，输入维度是所有卷积通道数之和（因为后面会拼接），输出 2 类（如正面/负面情感）
        self.decoder = nn.Linear(sum(num_channels), 2)



    def forward(self, inputs):
        # 每个嵌入层的输出形状都是（批量大小，词元数量，词元向量维度）连结起来
        # inputs(B,Q)
        # (B,Q,E)+(B,Q,E)=>(B,Q,2E)
        embeddings = torch.cat((
            self.embedding(inputs), 
            self.constant_embedding(inputs)
            ), dim=2) # 沿着向量维度将两个嵌入层连结起来

        # 根据一维卷积层的输入格式，重新排列张量
        # embeddings(B,Q,2E)=>(B,2E,Q)
        embeddings = embeddings.permute(0, 2, 1)

        # 对每个卷积层：卷积 -> ReLU -> 全局最大池化 -> squeeze 去掉最后一维
        pooled_list = []
        # embeddings(B,2E,Q)
        for conv in self.convs:
            # embeddings(B,2E,Q)
            # kernal(c,k)=(100,3)
            # conv_out(B,c,Q-k+1)=(B,100,Q-3+1)
            conv_out = conv(embeddings)
            # relu_out(B,c,Q-k+1)
            relu_out = self.relu(conv_out) 
            # pooled(B,c,1)=(B,100,1)
            pooled = self.pool(relu_out) 
            # pooled(B,c)=(B,100)
            squeezed = torch.squeeze(pooled, dim=-1) 
            # List(tensor(B,100))
            pooled_list.append(squeezed)

        # encoding(B,300)
        encoding = torch.cat(pooled_list, dim=1) 
        # encoding(B,300)
        # dropout(B,300)
        # decoder(B,2)
        outputs = self.decoder(self.dropout(encoding))
        return outputs

让我们创建一个textCNN实例。它有3个卷积层，卷积核宽度分别为3、4和5，均有100个输出通道。


In [ ]:
# 我们加载预训练的100维GloVe嵌入作为初始化的词元表示。
# 这些词元表示（嵌入权重）在`embedding`中将被训练
# 在`constant_embedding`中将被固定
glove_embedding = d2l.TokenEmbedding('glove.6b.100d')
pretrained_embedding = glove_embedding[vocab.idx_to_token]

# kernal:3x3、4x4、5x5
# 每种kernal有100个，对应100个输出通道
embed_size, kernel_sizes, nums_channels = 100, [3, 4, 5], [100, 100, 100]
devices = d2l.try_all_gpus()
net = TextCNN(len(vocab), pretrained_embedding, embed_size, kernel_sizes, nums_channels)

def init_weights(m):
    if type(m) in (nn.Linear, nn.Conv1d):
        nn.init.xavier_uniform_(m.weight)

net.apply(init_weights);

### 加载预训练词向量

与 :numref:`sec_sentiment_rnn`相同，我们加载预训练的100维GloVe嵌入作为初始化的词元表示。这些词元表示（嵌入权重）在`embedding`中将被训练，在`constant_embedding`中将被固定。


### 训练和评估模型

现在我们可以训练textCNN模型进行情感分析。


In [ ]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction="none")
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

In [ ]:
def predict_sentiment(net, vocab, sequence):
    """预测文本序列的情感"""
    sequence = torch.tensor(vocab[sequence.split()], device=d2l.try_gpu())
    # score(B,2)=(1,2)
    score = torch.softmax(net(sequence.reshape(1, -1)), dim=1)
    # label=(1,)
    # 在维度1上取最大值的"索引"，作为预测的类别（0=negative，1=positive）
    label = torch.argmax(score, dim=1).item()
    return score, label

In [ ]:
test_sentences = ['I love this movie very much',
                  'Every day is a new opportunity to grow',
                  'I believe in my ability',
                  'Good things are coming my way',
                  'I just like the way this movie is going',
                  'Let’s make a better world together',
                  'I am grateful for all the blessings in my life',
                  'Nothing ever goes right for me',
                  'I feel like I’m stuck and can’t move forward',
                  'No matter how hard I try, it’s never enough',
                  'I’m tired of facing disappointment after disappointment',
                  'It seems like the weather is bad']
with torch.no_grad():
    for sentence in test_sentences:
        score, label = predict_sentiment(net, vocab, sentence.lower())
        print(f'{sentence:<60} emotion [negative:{score[0][0]*100:.2f}%, positive:{score[0][1]*100:.2f}%]')

下面，我们使用训练好的模型来预测两个简单句子的情感。


In [ ]:
predict_sentiment(net, vocab, 'this movie is so great')

In [ ]:
predict_sentiment(net, vocab, 'this movie is so bad')

## 小结

* 一维卷积神经网络可以处理文本中的局部特征，例如$n$元语法。
* 多输入通道的一维互相关等价于单输入通道的二维互相关。
* 最大时间汇聚层允许在不同通道上使用不同数量的时间步长。
* textCNN模型使用一维卷积层和最大时间汇聚层将单个词元表示转换为下游应用输出。

## 练习

1. 调整超参数，并比较 :numref:`sec_sentiment_rnn`中用于情感分析的架构和本节中用于情感分析的架构，例如在分类精度和计算效率方面。
1. 请试着用 :numref:`sec_sentiment_rnn`练习中介绍的方法进一步提高模型的分类精度。
1. 在输入表示中添加位置编码。它是否提高了分类的精度？


[Discussions](https://discuss.d2l.ai/t/5720)
